# Goal: Build `bhm` in energy space

The problem here is that I can't convert directly from `bhm` in time to energy space because then the energy bins are different (17% range of variation) due to different distances from the fission chamber. Thus, I need to go back to the original construction of the `bicorr_hist_master` and create two versions: one in time, and one in energy. 

This is probably pretty hasty, but I am going to only create this for $nn$ events. Why you might ask? Because.

I'm choosing to not modify the original notebook `build_bicorr_hist_master` because... just because. I will work with the same methods here. 

I will work with the data in the `fnpc > datar` folder.

In [1]:
import matplotlib.pyplot as plt
import matplotlib.colors
import numpy as np
import os
import scipy.io as sio
import sys
import time
import inspect
import pandas as pd
from tqdm import *

# Plot entire array
np.set_printoptions(threshold=np.nan)

In [2]:
import seaborn as sns
sns.set_palette('spectral')

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
sys.path.append('../scripts/')
import bicorr as bicorr
import bicorr_plot as bicorr_plot
import bicorr_e as bicorr_e

# Step 1) Load the data from `bicorr1`

In [5]:
os.listdir('../datar/1')

['bicorr1', 'bicorr1_part', 'cced1', 'fig', 'timeOffset.txt']

In [6]:
with open('../datar/1/bicorr1_part') as f:
    print(f.read())

22  18  1  57.5  19  2  2.8438
39  17  2  2.9688  18  1  -244.38
50  17  2  8.6406  18  2  2.8438
65  17  2  13.156  27  2  4.0312
91  18  2  2.7812  27  2  4.1562
121  19  2  276.0  20  2  3.9375
123  22  2  6.2656  23  2  3.168
128  24  1  36.406  26  2  2.8125
134  18  2  2.5625  21  1  42.938
134  18  2  2.5625  24  1  55.312
134  21  1  42.938  24  1  55.312
139  24  2  3.0117  26  2  3.2656
144  19  2  -231.25  29  2  3.8047
153  27  1  40.875  28  1  57.875
159  27  1  -137.62  29  1  59.438
171  2  2  3.5273  12  2  29.812
174  4  2  4.2773  6  2  6.1875
189  20  1  58.625  30  2  2.2578
190  7  2  109.38  9  1  32.875
190  7  2  109.38  31  2  189.38
190  9  1  32.875  31  2  189.38
192  7  2  19.578  12  2  90.25
194  6  2  4.1562  31  2  3.0781
197  2  1  53.312  10  2  2.0625
198  3  2  10.586  4  1  35.656
198  3  2  10.586  26  2  -36.0
198  4  1  35.656  26  2  -36.0
225  19  2  146.0  29  2  2.8359
230  26  2  4.7656  29  1  46.719
234  4  1  45.75  12  2  4.0078
237  3

To remind ourselves what this file contains, the columns are:

* col 1) Event number
* col 2) d1ch
* col 3) d1 particle type
* col 4) d1 $\Delta t_1$
* col 5) d2ch
* col 6) d2 particle type
* col 7) d2 $\Delta t_2$

From this I need to calculate the energies. I don't really want to regenerate the `bicorr` file, or even the `bhm` file. I need a separate function that will take the `bicorr` file and generate a `bhm_e` distribution. 

In [5]:
bicorr_data = bicorr.load_bicorr(1, root_path = '../datar')

In [8]:
type(bicorr_data)

numpy.ndarray

I used a numpy array. That's kind of a shame. If I had used a pandas array, I could easily add new colums with energies, but oh well. Moving on.

Skipping step 2 to keep this notebook in line with `build_bicorr_hist_master.ipynb`.

# Step 3) Preallocate `bhm_e` matrix

Follow the method in `build_bicorr_hist_master`. 

In [14]:
help(bicorr_e.build_energy_bin_edges)

Help on function build_energy_bin_edges in module bicorr_e:

build_energy_bin_edges(e_min=0, e_max=15, e_step=0.025, print_flag=False)
    Construct energy_bin_edges for the two-dimensional bicorrelation energy histograms. 
    
    Use as: e_bin_edges, num_e_bins = bicorr.build_energy_bin_edges()
    
    Parameters
    ----------
    e_min : int, optional
        Lower energy boundary
    e_max : int, optional
        Upper energy boundary
    e_step : float, optional
        Energy bin size
    print_flag : bool, optional
        Whether to print array details
        
    Returns
    -------
    e_bin_edges : ndarray
        One-dimensional array of energy bin edges
    num_e_bins : ndarray
        Number of bins in energy dimension



In [6]:
e_bin_edges, num_e_bins = bicorr_e.build_energy_bin_edges()

In [11]:
print(e_bin_edges)
print(num_e_bins)

[  0.      0.025   0.05    0.075   0.1     0.125   0.15    0.175   0.2
   0.225   0.25    0.275   0.3     0.325   0.35    0.375   0.4     0.425
   0.45    0.475   0.5     0.525   0.55    0.575   0.6     0.625   0.65
   0.675   0.7     0.725   0.75    0.775   0.8     0.825   0.85    0.875
   0.9     0.925   0.95    0.975   1.      1.025   1.05    1.075   1.1
   1.125   1.15    1.175   1.2     1.225   1.25    1.275   1.3     1.325
   1.35    1.375   1.4     1.425   1.45    1.475   1.5     1.525   1.55
   1.575   1.6     1.625   1.65    1.675   1.7     1.725   1.75    1.775
   1.8     1.825   1.85    1.875   1.9     1.925   1.95    1.975   2.
   2.025   2.05    2.075   2.1     2.125   2.15    2.175   2.2     2.225
   2.25    2.275   2.3     2.325   2.35    2.375   2.4     2.425   2.45
   2.475   2.5     2.525   2.55    2.575   2.6     2.625   2.65    2.675
   2.7     2.725   2.75    2.775   2.8     2.825   2.85    2.875   2.9
   2.925   2.95    2.975   3.      3.025   3.05    3.075   3.1 

## Interaction type bins

In [17]:
# Number of bins in interaction type
num_intn_types = 1 #(0=nn, 1=np, 2=pn, 3=pp), only going to use nn

## Detector pair bins

In [15]:
# What are the unique detector numbers? Use same technique as in bicorr.py
chList, fcList, detList, num_dets, num_det_pairs = bicorr.build_ch_lists(print_flag=True)

Fission chamber channels: [ 0 16 32]
Detector channels: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 17 18 19 20 21 22 23 24 25 26
 27 28 29 30 31 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
Number of detectors: 45
Number of detector pairs: 990


## Preallocate matrix

In [14]:
bhm_e = np.zeros((num_det_pairs,num_intn_types,num_e_bins,num_e_bins),dtype=np.uint32)

In [15]:
bhm_e.shape

(990, 1, 600, 600)

How large when stored to disk?

In [16]:
bhm_e.nbytes/1e9

1.4256

This is pretty small. Good. I could even avoid converting it to and from sparse matrix at this size.

## Functionalize this

In [50]:
help(bicorr_e.alloc_bhm_e)

Help on function alloc_bhm_e in module bicorr_e:

alloc_bhm_e(num_det_pairs, num_intn_types, num_e_bins)
    Preallocate bhm_e
    
    Four dimensions: num_det_pairs x num_intn_types x num_e_bins x num_e_bins
    Interaction type index:  (0=nn, 1=np, 2=pn, 3=pp)
    
    Parameters
    ----------
    num_det_pairs : int
    num_intn_types : int, optional
    num_e_bins : int
    
    Returns
    -------
    bhm_e : ndarray
        Zero-filled bhm_e in energy space



In [18]:
bhm_e = bicorr_e.alloc_bhm_e(num_det_pairs, num_intn_types, num_e_bins)

In [18]:
bhm_e.shape

(990, 1, 600, 600)

# Step 4) Fill the histogram

Now I'm going to use the final method from `build_bhm` to fill `bhm_e` element by element.

I will need to add one more step, which is to retrieve the distance from each detector to the fission chamber for use in calculating the energy to each detector. Make it happen. 

## Set up a dictionary for retrieving detector distance

I want to call `dict_dist(det_num)` and have it return the detector distance in m.

First, load the file with detector distances. 

In [22]:
dict_det_dist = bicorr_e.build_dict_det_dist()

In [21]:
dict_det_dist

{1: 1.0649999999999999,
 2: 1.0509999999999999,
 3: 1.038,
 4: 1.0329999999999999,
 5: 1.0249999999999999,
 6: 1.0290000000000001,
 7: 1.0449999999999999,
 8: 1.0640000000000001,
 9: 1.0840000000000001,
 10: 1.073,
 11: 1.056,
 12: 1.042,
 13: 1.04,
 14: 1.038,
 15: 1.0429999999999999,
 17: 1.0509999999999999,
 18: 1.0680000000000001,
 19: 1.083,
 20: 1.087,
 21: 1.0680000000000001,
 22: 1.0680000000000001,
 23: 1.0680000000000001,
 24: 1.0680000000000001,
 25: 1.0680000000000001,
 26: 1.0859999999999999,
 27: 1.093,
 28: 1.103,
 29: 1.0580000000000001,
 30: 1.0469999999999999,
 31: 1.028,
 33: 1.0249999999999999,
 34: 1.018,
 35: 1.0270000000000001,
 36: 1.0429999999999999,
 37: 1.056,
 38: 1.0840000000000001,
 39: 1.0680000000000001,
 40: 1.0529999999999999,
 41: 1.0390000000000001,
 42: 1.032,
 43: 1.03,
 44: 1.0429999999999999,
 45: 1.05,
 46: 1.0629999999999999,
 47: 1.0840000000000001}

In [22]:
dict_det_dist[45]

1.05

## Set up dictionaries for returning pair and type indices

In [20]:
# Set up dictionary for returning detector pair index
det_df = bicorr.load_det_df()
dict_pair_to_index, dict_index_to_pair, dict_pair_to_angle = bicorr.build_dict_det_pair(det_df)

In [24]:
print(det_df)
print(dict_pair_to_index)
print(dict_index_to_pair)
print(dict_pair_to_angle)

     d1  d2  d1d2       angle
0     1   2   102   15.000000
1     1   3   103   30.000000
2     1   4   104   45.000000
3     1   5   105   60.000000
4     1   6   106   75.000000
5     1   7   107   90.000000
6     1   8   108  105.000000
7     1   9   109  120.000000
8     1  10   110   16.328077
9     1  11   111   24.647971
10    1  12   112   37.234312
11    1  13   113   51.009565
12    1  14   114   65.207372
13    1  15   115   79.577644
14    1  17   117   94.005814
15    1  18   118  108.412595
16    1  19   119  122.706267
17    1  20   120   31.604677
18    1  21   121   40.871807
19    1  22   122   52.473028
20    1  23   123   65.128626
21    1  24   124   78.266001
22    1  25   125   91.587592
23    1  26   126  104.885639
24    1  27   127  117.941468
25    1  28   128  130.414743
26    1  29   129   60.000000
27    1  30   130   75.000000
28    1  31   131   90.000000
29    1  33   133  105.000000
..   ..  ..   ...         ...
960  39  46  3946  105.000000
961  39  4

In [99]:
# Type index
dict_type_to_index = {11:0, 12:1, 21:2, 22:3}

## Calculate energy for one event

In [25]:
i = 3

In [26]:
bicorr_data[i]

(44, 29, 1, 40.84375, 37, 1, 46.0625)

In [28]:
det1dist = dict_det_dist[bicorr_data[i]['det1ch']]
det2dist = dict_det_dist[bicorr_data[i]['det2ch']]
print(det1dist,det2dist)

1.058 1.056


These are pretty close together. Now convert those to energy using the time stamps. Only proceed when both time stamps are greater than 0. 

In [29]:
det1t = bicorr_data[i]['det1t']
det2t = bicorr_data[i]['det2t']
print(det1t, det2t)

40.844 46.063


In [30]:
det1e = bicorr.convert_time_to_energy(det1t, det1dist)
det2e = bicorr.convert_time_to_energy(det2t, det2dist)
print(det1e,det2e)

3.52593721684 2.76176857758


Set up info for filling the histogram

In [32]:
e_min = np.min(e_bin_edges); e_max = np.max(e_bin_edges)
e_step = e_bin_edges[1]-e_bin_edges[0]

Only proceed if both particles are neutrons AND both times are greater than 0. How do I implement this logic?

In [86]:
i = 16
event = bicorr_data[i]
det1t = event['det1t']; det2t = event['det2t'];

print(event, det1t, det2t, event['det1par'], event['det2par'])

(207, 36, 2, 89.375, 40, 2, 5.9296875) 89.375 5.9297 2 2


In [87]:
np.logical_and([det1t > 0, event['det1par'] == 1], [det2t>0, event['det2par'] == 1])

array([ True, False], dtype=bool)

The tricky thing here is that `np.logical_and` looks at elements 0 of both input arrays as one pair, then elements 1, etc. I had originally implemented it with the assumption that it looked at each input array as a pair. Thus, the split implementation.

Implement `tqdm` status bar. 

In [107]:
for i in tqdm(np.arange(bicorr_data.shape[0]),ascii=True,disable=False):
    event = bicorr_data[i]
    det1t = event['det1t']; det2t = event['det2t'];
    
    logic = np.logical_and([det1t > 0, event['det1par'] == 1], [det2t>0, event['det2par'] == 1])
    
    if np.logical_and(logic[0],logic[1]): # nn with both t > 0
        det1dist = dict_det_dist[event['det1ch']]
        det2dist = dict_det_dist[event['det2ch']]
        
        det1e = bicorr.convert_time_to_energy(det1t, det1dist)
        det2e = bicorr.convert_time_to_energy(det2t, det2dist)
        
        # Check that they are in range of the histogram        
        if np.logical_and(e_min < det1e < e_max, e_min < det2e < e_max):
            # Determine index of detector pairs
            pair_i = dict_pair_to_index[event['det1ch']*100+event['det2ch']]
        
            # Determine indices of energy values
            e1_i = int(np.floor((det1e-e_min)/e_step))
            e2_i = int(np.floor((det2e-e_min)/e_step))
            
            # Increment bhm_e
            pair_i = dict_pair_to_index[event['det1ch']*100+event['det2ch']]
            bhm_e[pair_i,0,e1_i,e2_i] += 1

100%|##########| 695/695 [00:00<00:00, 16752.05it/s]


## Functionalize it

In [119]:
import inspect

In [121]:
print(inspect.getsource(bicorr_e.fill_bhm_e))

def fill_bhm_e(bhm_e, bicorr_data, det_df, e_bin_edges, disable_tqdm = False):
    """
    Fill bhm_e. Structure:
        Dimension 0: detector pair, use dictionary 'dict_pair_to_index', where pair is (100*det1ch+det2ch)
        Dimension 1: interaction type, length 1. Only storing 0=nn.
        Dimension 2: e bin for detector 1
        Dimension 3: e bin for detector 2
        
    Must have allocated bhm_e and loaded bicorr_data
    
    Parameters
    ----------
    bhm_e : ndarray
        Master histogram of bicorrelation events in energy space. Dimensions listed above.
    bicorr_data : ndarray
        Each element contains the following info for one bicorrelation pair
        Columns are 0: event, np.int32
                    1: det1ch, np.int8
                    2: det1par, np.int8
                    3: det1t, np.float16
                    4: det2ch, np.int8
                    5: det2par, np.int8
                    6: det2t, np.float16
    det_df : pandas dataFrame
        

In [23]:
bhm_e = bicorr_e.alloc_bhm_e(num_det_pairs, num_intn_types, num_e_bins)
bhm_e = bicorr_e.fill_bhm_e(bhm_e, bicorr_data, det_df, dict_det_dist, e_bin_edges, disable_tqdm = False)

100%|##########| 695/695 [00:00<00:00, 15434.77it/s]


Skipping step 5. I am not going to convert to sparse matrix because the file size will be small anyway.

In [152]:
bhm_e.shape

(990, 1, 600, 600)

# Step 6) Save the histogram and related vectors to disk

What do I need to save? Mostly the same stuff but in energy units.

In [24]:
save_filename = r'../datar/1/bhm_e'
note = 'Here is my note'

np.savez(save_filename, bhm_e = bhm_e, e_bin_edges=e_bin_edges, note = note)

In [8]:
bicorr_e.save_bhm_e(bhm_e, e_bin_edges, r'../datar/1/')

# Step 7) Reload from disk

In [5]:
load_filename = r'../datar/1/bhm_e.npz'

bhm_e = np.load(load_filename)['bhm_e']
e_bin_edges = np.load(load_filename)['e_bin_edges']
note = np.load(load_filename)['note']

In [8]:
print(bhm_e.shape)
print(e_bin_edges.shape)
print(note)

(990, 1, 600, 600)
(601,)
note


In [7]:
bhm_e, e_bin_edges, note = bicorr_e.load_bhm_e(r'../datar/1/')

In [9]:
help(bicorr_e.save_bhm_e)

Help on function save_bhm_e in module bicorr_e:

save_bhm_e(bhm_e, e_bin_edges, save_folder=None, bhm_e_filename='bhm_e', note='note')
    Save bhm_e to .npz file. (Reload using load_bhm_e function)
    
    Parameters
    ----------
    bhm_e : ndarray
        Master histogram of bicorrelation events in energy space. 
        Dimension 0: detector pair, use dictionary 'dict_pair_to_index', where pair is (100*det1ch+det2ch)
        Dimension 1: interaction type, length 1. Only storing 0=nn.
        Dimension 2: e bin for detector 1
        Dimension 3: e bin for detector 2
    e_bin_edges : ndarray
        One-dimensional array of energy bin edges
    save_folder : str, optional
        Optional destination folder. If None, then save in current working directory
    bhm_e_filename : str, optional
        Filename
    note : str, optional
        Note to include
    
    Returns
    -------
    n/a



# Functionalize for many folders

I need to pull the `bicorr` files from many folders and produce `bhm_e` along with `bhm`.

If I were going to reproduce `bhm` from the beginning, I would modify `build_bhm` to include another line of generating `bhm_e`. In this case, though, I am only going to produce `bhm_e` so I will write a separate function. 

In [5]:
help(bicorr_e.build_bhm_e)

Help on function build_bhm_e in module bicorr_e:

build_bhm_e(folder_start=1, folder_end=2, det_df=None, dict_det_dist=None, e_bin_edges=None, checkpoint_flag=True, save_flag=True, bhm_e_filename='bhm_e', root_path=None, disable_tqdm=False, print_flag=True, return_flag=True)
    Load bicorr_data from folder's bicorr# file and fill energy histogram. Loop through folders specified by `folder_start` and `folder_end`. Built for e_bin_edges generated using default settings in bicorr.build_energy_bin_edges().
    
    Parameters
    ----------
    folder_start : int, optional
        First folder
    folder_end : int, optional
        Last folder + 1 (for example, folder_end = 2 will end at folder 1)
    det_df : pandas dataFrame, optional
        dataFrame of detector pair indices and angles   
        Default is to look for the file in '../meas_info/det_df_pairs_angles.csv'
    dict_det_dist : dict
        Dictionary of detector channel number : distance from fc
        Default is to look 

In [9]:
bhm_e, e_bin_edges = bicorr_e.build_bhm_e(1,3,root_path = '../datar/')

Generating bicorr histogram for bicorr data in folders:  [1 2]
Loading data in folder  1
Building bhm in folder  1


100%|##########| 695/695 [00:00<00:00, 21774.19it/s]


Loading data in folder  2
Building bhm in folder  2


100%|##########| 695/695 [00:00<00:00, 25810.30it/s]


Saving bhm_e to .npz file
Bicorr hist master bhm_e build complete


In [5]:
help(bicorr_e.load_bhm_e)

Help on function load_bhm_e in module bicorr_e:

load_bhm_e(filepath=None, filename=None)
    Load .npz file containing `bhm_e`, `e_bin_edges`, and `note`. This file was probably generated by the function `save_bhm_e`.
    
    Parameters
    ----------
    filepath : str, optional
        Where is the `bhm_e.npz` file? If None, look in current directory
    filename : str, optional
        What is the file called? If None, then look for `bhm_e.npz`
    
    Returns
    -------
    bhm_e : ndarray
        Master histogram of bicorrelation events in energy space. 
        Dimension 0: detector pair, use dictionary 'dict_pair_to_index', where pair is (100*det1ch+det2ch)
        Dimension 1: interaction type, length 1. Only storing 0=nn.
        Dimension 2: e bin for detector 1
        Dimension 3: e bin for detector 2
    e_bin_edges : ndarray
        One-dimensional array of energy bin edges
    note : str, optional
        Note to include



In [10]:
bhm_e, e_bin_edges, note = bicorr_e.load_bhm_e()

In [11]:
note

array('bhm_e generated from folder 1 to 3 in directory C:\\Users\\pfsch\\Box Sync\\Projects\\fnpc\\methods', 
      dtype='<U93')

I call this a win. Moving on. 